In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Input, concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, BatchNormalization, concatenate



def multi_scale_block(input_tensor):
    conv_1x1 = Conv2D(filters=64, kernel_size=(1, 1), activation='relu', padding='same')(input_tensor)
    conv_3x3 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(input_tensor)
    conv_5x5 = Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding='same')(input_tensor)

    conv_1x1 = BatchNormalization()(conv_1x1)
    conv_3x3 = BatchNormalization()(conv_3x3)
    conv_5x5 = BatchNormalization()(conv_5x5)

    merged = concatenate([conv_1x1, conv_3x3, conv_5x5], axis=-1)
    return merged


input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))


x = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = multi_scale_block(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = multi_scale_block(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

x = Flatten()(x)

x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.1)(x)

output_layer = Dense(9, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


In [ ]:
model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
def lr_schedule(epoch):
    initial_lr = 1e-4
    k = 0.2  # Decay rate
    lr = initial_lr * np.exp(-k * epoch)
    print('Learning rate: ', lr)
    return lr